# Review notebook: harvest senses with provenance

Notebook for reviewing functions

- `get_provenance_by_semantic_class`
- `extend_from_saved_lemma_query`

These functions assume:
    - a pickled dataframe with information harvested

What these functions should do:
    - for a given lemma id (e.g. lemm

In [1]:
!git branch

  1-dataframe
  19-machine-tagger
  3-group-senses
* 4-semantic-provenance
  dev
  master
  oed-experiments


In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [4]:
from utils.dataset_download import *
import pickle
import json
from pathlib import Path, PosixPath
import pandas as pd

# Load credentials, set paths and arguments

In [5]:
# import API credentials
with open('oed_experiments/oed_credentials.json') as f:
    auth = json.load(f)

In [6]:
# define lemma
lemma_id = "machine_nn01"

In [7]:
dp = "../data"

In [8]:
save_path = Path(dp)
save_path.mkdir(exist_ok=True)

In [9]:
start,end = 1750,1950
lemma_id = 'machine_nn01'

## Run function

In [11]:
extended_df = extend_from_saved_lemma_query(auth,lemma_id,start,end)

Get all sense for the lemma machine_nn01



Get all synonyms of the senses listed in machine_nn01



Get all branches for seed senses and synonyms


In [14]:
extended_df.head(3)

,categories,daterange,definition,first_use,id,lemma,main_current_sense,meta,notes,oed_reference,oed_url,part_of_speech,provenance,provenance_type,quotation_ids,semantic_class_ids,semantic_class_last_id,transitivity,word_id
0,"{'topic': [], 'usage': [['rare']], 'region': []}","{'end': None, 'start': 1545, 'obsolete': False...","A material or immaterial structure, esp. the f...",J. Schäfer,machine_nn01-38473945,machine,False,"{'created': 1904, 'revised': True, 'updated': ...",[],"machine, n., sense I.1a",https://www.oed.com/view/Entry/111850#eid38473945,NN,"[[machine_nn01-38473945, seed, machine_nn01]]",seed,"[machine_nn01-38473950, machine_nn01-38473961,...","[[1, 111290, 118635, 119024, 120162, 120172], ...","[120172, 120173]",None,machine_nn01
1,"{'topic': [['Military', 'Weaponry']], 'usage':...","{'end': None, 'start': 1583, 'obsolete': False...",A military engine or siege-tower. Cf. war mach...,Brian Melbancke,machine_nn01-38474233,machine,False,"{'created': 1904, 'revised': True, 'updated': ...",[],"machine, n., sense II.3",https://www.oed.com/view/Entry/111850#eid38474233,NN,"[[machine_nn01-38474233, seed, machine_nn01]]",seed,"[machine_nn01-38474243, machine_nn01-38474252,...","[[153072, 160439, 163207, 163208, 163377, 1633...",[163378],None,machine_nn01
2,"{'topic': [], 'usage': [], 'region': []}","{'end': 1707, 'start': 1595, 'obsolete': True,...",spec. A scheme or plot. Obsolete.,Elizabeth I,machine_nn01-38474097,machine,False,"{'created': 1904, 'revised': True, 'updated': ...",[],"machine, n., sense I.1b",https://www.oed.com/view/Entry/111850#eid38474097,NN,"[[machine_nn01-38474097, seed, machine_nn01]]",seed,"[machine_nn01-38474102, machine_nn01-38474122,...","[[1, 84689, 87987, 87988, 87989, 88083, 88109,...",[88126],None,machine_nn01


In [13]:
extended_df.shape

(8383, 19)

# Inspect function

In [ ]:
def get_provenance_by_semantic_class(row):
    """
    decide on the relation between the sense and the target querry
    here we use the lowest semantic class id to decide on the relation
    
    if last semantic class id (sc_ids[-1]) == provenance id: then sense is sibling of provenance id
    elif provenance semantic class id in the list of semantic class last ids
    (but provenance not the last one): then sense is descendant of provenance id
    Argument:
        row (pd.Series): row of dataframe obtained from branchsenses endpoint
    
    Returns:
        nested listed in the format of [lowest semantic class id, relation, provenance semantic class id]
            in other words it said that for a given sense (which can have multiple semantic class ids)
            the lowest semantic class id stands in the relation "sibling" or "descendant" of the 
            provenance semantic class id
    """
    
    provenance = []
    
    for sc_ids in row.semantic_class_ids:
        relation = ''
        
        # scenario 1
        if sc_ids[-1] == row.provenance_pivot:
            relation = 'sibling'
        
        # scenario 2
        elif (row.provenance_pivot in sc_ids):
            relation = 'descendant'
        
        # exclude other relation
        if relation:
            provenance.append([sc_ids[-1], relation, row.provenance_pivot])
    
    if not provenance:
        print(f'No descendants or siblings found for {row.id}')
 
    return provenance


In [ ]:
# helper function to get last element in a nested list
get_last_id = lambda nested_list :[l[-1] for l in nested_list]
    
# load seed query dataframe
query_df = pd.read_pickle(f"./data/senses_{lemma_id}.pickle")
    
# use the sense endpoint to ensure all information 
# can be properly concatenated in one dataframe
    
# retrieve all sense ids
query_sense_ids = query_df.id.unique()

In [ ]:
# get all senses by sense id
print(f"Get all sense for the lemma {lemma_id}")
seeds = [(s,query_oed(auth,'sense',s,
                flags=f"current_in='{start}-{end}'&limit=1000", # probably "current_in" not needed here see APi
                verbose=False)) # set verbose to True to see the url request
                    for s in tqdm(query_sense_ids)]

In [ ]:
# convert to dataframe
seeds_df = pd.DataFrame([seed['data'] for s_id,seed in seeds])

# seed_df contains all the senses of the word machine_nn01
# we distinguish between provenance and provenance_type
# provenance will refer to specific word, sense of semantic class ids
# provenance_type will distinguish between different types of extension
# define provenance, these words are "seed"
seeds_df['provenance'] = [[[i,'seed',lemma_id]] for i in seeds_df.id] # for the seed sense we use the id of the word machine_nn0
                                       # we use list here, reason is explained later, see provenance of synonyms
seeds_df['provenance_type'] = 'seed' # categorize these lemmas as seed

In [ ]:
# get all synonyms for the seed senses
# reminder synonyms uses same function as the /senses/ endpoint, flags should work here
print(f"Get all synonyms of the senses listed in {lemma_id}")
synonyms = [(s,query_oed(auth,'sense',s,
                level='synonyms',
                flags=f"current_in='{start}-{end}'&limit=1000"))
                        for s in tqdm(query_sense_ids)]

In [ ]:
# transform list of synonyms to a dataframe
synonyms_df = pd.DataFrame([s for s_id,syn in synonyms for s in syn['data']])
    
# for synonyms the provenance_type is set to "synonym"
synonyms_df['provenance_type'] = 'synonym'
# for synonyms we refer the sense_id via which this synonym was retrieved
synonyms_df['provenance'] = [[[s['id'],'synonym',s_id]] for s_id,syn in synonyms for s in syn['data']]

In [ ]:
# seed + synonyms constitute the nucleas of our query
# these are saved in the core_df
# shape should be 485 (synonyms senses) + 26 (seed senses)
core_df = pd.concat([seeds_df,synonyms_df],sort=True)
    
# branch out from there
# we save the lowest level of the semantic_class_last_id columns
core_df['semantic_class_last_id'] = core_df['semantic_class_ids'].apply(get_last_id)

In [ ]:
# retrieve all the _lowest_ (or last) semantic class ids for the core senses so far
semantic_class_ids = set([s for l in core_df.semantic_class_last_id.to_list() for s in l])

In [ ]:
# now, we use the descendants endpoint
# for each lowest semantic class id
# we get all "descendants" which according the API documentation
# returns an array of senses that belong to the semantic class
# specified by ID, plus senses that belong to its child and descendant classes.
print("Get all branches for seed senses and synonyms")
branches = [(idx,query_oed(auth,'semanticclass', idx, 
                        level='branchsenses', # 
                        flags=f"current_in='{start}-{end}'&limit=1000"))
                            for idx in tqdm(semantic_class_ids)]

In [ ]:
# convert API response to dataframe
branches_df = pd.DataFrame([s for idx,branch in branches for s in branch['data']])
    
# ISSUE: again we have duplicate 
# senses here, as some appear multiple time as
# in the same semantic class (or as descendant)
    
# provenance_type is branch with semantic class id 
# that was use for retrieving the sense is the provenance
branches_df['provenance_type'] = 'branch'
    
# we create a provenance_pivot columsn, which shows
# the semantic class id via which the sense was retrieved
branches_df['provenance_pivot'] = [idx for idx, branch in branches for s in branch['data']]

In [ ]:
# now there are two scenarios to specify for the pro
# both scenarios can apply to one sense
# if last semantic class id (sc_ids[-1]) == provenance id: then sense is sibling of provenance id
# elif provenance semantic class id in the list of semantic class last ids
# (but provenance not the last one): then sense is descendant of provenance id
    
branches_df['provenance'] = branches_df.apply(get_provenance_by_semantic_class,axis=1)
    
# drop the provenance_pivot column
branches_df.drop('provenance_pivot',axis=1,inplace=True)
    
# concatenate core and branch senses
# ISSUE: have a closer look at the warning message
extended_df = pd.concat([core_df,branches_df],sort=True)

# to check if rows match
#extended_df.shape[0] == core_df.shape[0] + branches_df.shape[0]
# save dataframe as pickle
extended_df.to_pickle(f"./data/extended_{lemma_id}.pickle") 

# Get machine_nn01 senses

In [ ]:
#query the API and get the json response
sense_json = query_oed(credentials,'word',lemma_id,flags='include_senses=true&include_quotations=true')

# convert the json in a dataframe
senses_df = convert_json_to_dataframe(sense_json)

In [ ]:
# save the dataframe
# as pickle
senses_df.to_pickle(save_path / f"senses_{lemma_id}.pickle")
# as csv
senses_df.to_csv(save_path / f"senses_{lemma_id}.tsv",sep='\t')

In [ ]:
# # open pickle file to avoid calling the API again
# with open(save_path / f"senses_{lemma_id}.pickle",'rb') as in_pickle:
#     machine_senses_df = pickle.load(in_pickle)

In [ ]:
# # get all senses that are siblings and descendants
# # of the semantic class of senses listed in previously obtained query 
# responses = traverse_thesaurus(credentials,machine_senses_df)

In [ ]:
# # traverse tree or load responses 
# # responses = traverse_thesaurus(credentials,machine_senses_df)
# with open(f'{dp}/tree_traversal.pickle','rb') as in_pickle:
#     responses = pickle.load(in_pickle)

In [ ]:
# # get all quoations for the senses in the responses variable
# quotations = get_quotations_from_thesaurus(credentials,responses)

In [ ]:
# # merge and save all information stored in the seperate pickle files
# df = merge_pickled(Path("./data/senses_machine_nn01.pickle"),
#                    Path("./data/tree_traversal.pickle"),
#                    Path("./data/tree_traversal_quotations.pickle"))

In [ ]:
# df.to_pickle(f"{dp}/{lemma_id}_all.pickle")

In [ ]:
# df.to_pickle(f"{dp}/{lemma_id}_all.pickle")

## Start Expansion

In [ ]:
start,end = 1750,1950
lemma_id = 'machine_nn01'

In [ ]:
get_last_id = lambda nested_list :[l[-1] for l in nested_list]

In [ ]:
query_df = pd.read_pickle(f"./data/senses_{lemma_id}.pickle")
    
# use the sense endpoint to ensure all information 
# can be properly concatenated in one dataframe
    
# retrieve all sense ids
query_sense_ids = query_df.id.unique()

In [ ]:
query_sense_ids

In [ ]:
auth = credentials# for code checking, remove later

In [ ]:
print(f"Get all sense for the lemma {lemma_id}")
seeds = [(s,query_oed(auth,'sense',s,
                    flags=f"current_in='{start}-{end}'&limit=1000", # probably "current_in" not needed here see APi
                      verbose=False)) # set verbose to True to see the url request
                        for s in tqdm(query_sense_ids)]

In [ ]:
# convert to dataframe
seeds_df = pd.DataFrame([seed['data'] for s_id,seed in seeds])
seeds_df.head(3)

In [ ]:
# seed_df contains all the senses of the word machine_nn01
# we distinguish between provenance and provenance_type
# provenance will refer to specific word, sense of semantic class ids
# provenance_type will distinguish between different types of extension
# define provenance, these words are "seed"
seeds_df['provenance'] = [[[i,'seed',lemma_id]] for i in seeds_df.id] # for the seed sense we use the id of the word machine_nn0
                                       # we use list here, reason is explained later, see provenance of synonyms
seeds_df['provenance_type'] = 'seed' # categorize these lemmas as seed

In [ ]:
# get all synonyms for the seed senses
# reminder synonyms uses same function as the /senses/ endpoint, flags should work here
print(f"Get all synonyms of the senses listed in {lemma_id}")
synonyms = [(s,query_oed(auth,'sense',s,
                level='synonyms',
                flags=f"current_in='{start}-{end}'&limit=1000"))
                        for s in tqdm(query_sense_ids)]



In [ ]:
# transform list of synonyms to a dataframe
synonyms_df = pd.DataFrame([s for s_id,syn in synonyms for s in syn['data']])

In [ ]:
# for synonyms the provenance_type is set to "synonym"
synonyms_df['provenance_type'] = 'synonym'
# for synonyms we refer the sense_id via which this synonym was retrieved
synonyms_df['provenance'] = [[[s['id'],'synonym',s_id]] for s_id,syn in synonyms for s in syn['data']]

In [ ]:
# remove senses from synonyms that already appear in the seed_df
# otherwise we'd have different types of provenance
# for the same sense id
# print(synonyms_df.shape)
# synonyms_df = synonyms_df.loc[~synonyms_df.id.isin(seeds_df.id)]
# print(synonyms_df.shape)

In [ ]:
# explaining the line  [s_id for s_id,syn in synonyms for s in syn['data']]
# remove later
# print(synonyms[0][0]) # print the sense_id
# print(synonyms[0][1]) # print synonyms for sense_id
# print(synonyms[0][1]['data']) # get 'data' for synonyms with send_id
# print(len(synonyms[0][1]['data'])) # number of senses
# synonym_sense_ids = [s_id for s_id,syn in synonyms for s in syn['data']]
# synonym_sense_ids[:10]

In [ ]:
# this should 485 synonyms
synonyms_df.provenance[0]

In [ ]:
# ISSUE: we have some duplicates, i.e. some senses appear multiple times as synonym 
# for the collection of seed_sense
# is this a problem?
# in the code below we keep duplicates, but allow provenance to list all senses
len(set(synonyms_df.id))

In [ ]:
# Counter(synonyms_df.id).most_common(5)

In [ ]:
#synonyms_df[synonyms_df.id=="machina_nn01-38472486"].provenance

In [ ]:
#pivot = synonyms_df.groupby(["id"],as_index=True)['provenance_pivot'].apply(list).to_frame(name='provenance')

In [ ]:
#synonyms_df = synonyms_df.merge(pivot,right_index=True,left_on='id',how='left')
#synonyms_df.shape

In [ ]:
#synonyms_df.drop('provenance_pivot',axis=1,inplace=True)

In [ ]:
# seed + synonyms constitute the nucleas of our query
# these are saved in the core_df
# shape should be 485 (synonyms senses) + 26 (seed senses)
core_df = pd.concat([seeds_df,synonyms_df])
core_df.shape

In [ ]:
# branch out from there
# we save the lowest level of the semantic_class_last_id columns
core_df['semantic_class_last_id'] = core_df['semantic_class_ids'].apply(get_last_id)
core_df.head(3)

In [ ]:
# retrieve all the _lowest_ (or last) semantic class ids for the core senses so far
semantic_class_ids = set([s for l in core_df.semantic_class_last_id.to_list() for s in l])
len(semantic_class_ids)

In [ ]:
# now, we use the descendants endpoint
# for each lowest semantic class id
# we get all "descendants" which according the API documentation
# returns an array of senses that belong to the semantic class
# specified by ID, plus senses that belong to its child and descendant classes.
print("Get all branches for seed senses and synonyms")
branches = [(idx,query_oed(auth,'semanticclass', idx, 
                        level='branchsenses', # 
                        flags=f"current_in='{start}-{end}'&limit=1000"))
                            for idx in tqdm(semantic_class_ids)]
    


In [ ]:
# convert API response to dataframe
branches_df = pd.DataFrame([s for idx,branch in branches for s in branch['data']])

In [ ]:
# ISSUE: again we have duplicate 
# senses here, as some appear multiple time as
# in the same semantic class (or as descendant)
print(branches_df.shape[0],len(set(branches_df.id)))

In [ ]:
# provenance_type is branch with semantic class id 
# that was use for retrieving the sense is the provenance
branches_df['provenance_type'] = 'branch'

In [ ]:
# we create a provenance_pivot columsn, which shows
# the semantic class id via which the sense was retrieved
branches_df['provenance_pivot'] = [idx for idx, branch in branches for s in branch['data']]

In [ ]:
#pivot = branches_df.groupby(["id"],as_index=True)['provenance_pivot'].apply(list).to_frame(name='provenance')

In [ ]:
#branches_df = branches_df.merge(pivot,right_index=True,left_on='id',how='left')
#branches_df.shape

In [ ]:
#branches_df.drop('provenance_pivot',axis=1,inplace=True)

In [ ]:
#branches_df['semantic_class_last_id'] = branches_df['semantic_class_ids'].apply(get_last_id)

In [ ]:
#branches_df.head()

In [ ]:


def get_provenance_by_semantic_class(row):
    """
    decide on the relation between the sense and the target querry
    here we use the lowest semantic class id to decide on the relation
    
    if last semantic class id (sc_ids[-1]) == provenance id: then sense is sibling of provenance id
    elif provenance semantic class id in the list of semantic class last ids
    (but provenance not the last one): then sense is descendant of provenance id
    Argument:
        row (pd.Series): row of dataframe obtained from branchsenses endpoint
    
    Returns:
        nested listed in the format of [lowest semantic class id, relation, provenance semantic class id]
            in other words it said that for a given sense (which can have multiple semantic class ids)
            the lowest semantic class id stands in the relation "sibling" or "descendant" of the 
            provenance semantic class id
    """

    provenance = []
    
    for sc_ids in row.semantic_class_ids:
        relation = ''
        
        
        if sc_ids[-1] == row.provenance_pivot:
            relation = 'sibling'
            
        elif (row.provenance_pivot in sc_ids):
            relation = 'descendant'
        
        if relation:
            provenance.append([sc_ids[-1], relation, row.provenance_pivot])
    
    if not provenance:
        print(f'No descendants or siblings found for {row.id}')
 
    return provenance



In [ ]:
branches_df['provenance'] = branches_df.apply(get_provenance_by_semantic_class,axis=1)

In [ ]:
# for idx in range(100,200):
#     print()
#     print(branches_df.iloc[idx]['semantic_class_ids'])
#     print(branches_df.iloc[idx]['semantic_class_last_id'])
#     print(branches_df.iloc[idx]['provenance_pivot'])
#     print(branches_df.iloc[idx]['provenance'])

In [ ]:
# Inspect function

In [ ]:
extended_df.to_pickle(f"./data/extended_{lemma_id}.pickle") 

In [ ]:
branches_df.drop('provenance_pivot',axis=1,inplace=True)

In [ ]:
# concatenate core and branch senses
# ISSUE: have a closer look at the warning message
extended_df = pd.concat([core_df,branches_df],sort=True)

In [ ]:
extended_df.shape[0] == core_df.shape[0] + branches_df.shape[0]

In [ ]:
extended_df = extend_from_lemma_query(credentials,lemma_id,start,end)

## Fin.

In [ ]:
def extend_from_lemma_query_depr(auth,lemma_id,start=1750,end=1950):
    """Extends senses from a dataframe generate from accessing
    the API via the word endpoint. The script first retrieves all
    senses, then synonyms for these senses, then other senses that 
    match the semantic classes of the retrieved senses.
    
    This script also aims to record the "provenance" of words, 
    their relation to the initial query, which can help to 
    select of filter words later on.
    
    Arguments:
        lemma_id (str)
        start (int)
        end (int)
    Returns
        a pandas.DataFrame
    """
    # load seed query dataframe
    query_df = pd.read_pickle(f"./data/senses_{lemma_id}.pickle")
    
    # use the sense endpoint to ensure all information 
    # can be properly concatenated in one dataframe
    
    # retrieve all sense ids
    query_sense_ids = query_df.id.unique()
    
    # get all senses by sense id
    print(f"Get all sense for the lemma {lemma_id}")
    seeds = [(s,query_oed(auth,'sense',s,
                    flags=f"current_in='{start}-{end}'&limit=1000"))
                        for s in tqdm(query_sense_ids)]
    
    # convert to dataframe
    seeds_df = pd.DataFrame([seed['data'] for s_id,seed in seeds])
    
    # define provenance, these words are "seed"
    seeds_df['provenance'] = seeds_df.id
    seeds_df['provenance_type'] = 'seed'
    
    # get all synonyms for the seed senses
    print(f"Get all synonyms of the senses listed in {lemma_id}")
    synonyms = [(s,query_oed(auth,'sense',s,
                    level='synonyms',
                    flags=f"current_in='{start}-{end}'&limit=1000"))
                            for s in tqdm(query_sense_ids)]

    # transform list of synonyms to a dataframe
    synonyms_df = pd.DataFrame([s for s_id,syn in synonyms for s in syn['data']])
    
    # these items have provenancy type "synonym"
    synonyms_df['provenance'] = [s_id for s_id,syn in synonyms for s in syn['data']]
    synonyms_df['provenance_type'] = 'synonym'
    
    # seed + synonyms constitute the nucleas of our query
    # branch from there
    core_df = pd.concat([seeds_df,synonyms_df])
    core_df['semantic_class_last_id'] = core_df['semantic_class_ids'].apply(get_last_id)
    
    # retrieve all semantic class ids for the senses so far
    semantic_class_ids = set([s for l in core_df.semantic_class_last_id.to_list() for s in l])

    # get all the branches for the retrieve semantic class ids
    print("Get all branches for seed senses and synonyms")
    branches = [(idx,query_oed(auth,'semanticclass', idx, 
                        level='branchsenses',
                        flags=f"current_in='{start}-{end}'&limit=1000"))
                            for idx in tqdm(semantic_class_ids)]
    
    # convert API response to dataframe
    branches_df = pd.DataFrame([s for idx,branch in branches for s in branch['data']])
    
    # provenance_type is branch with semantic class id 
    # that was use for retrieving the sense is the provenance
    branches_df['provenance'] = [idx for idx,branch in branches for s in branch['data']]
    branches_df['provenance_type'] = 'branch'
    
    branches_df['semantic_class_last_id'] = branches_df.semantic_class_ids.apply(get_last_id)
    
    # remove senses that already appear in the core_df
    branches_df_red = branches_df.loc[~branches_df.id.isin(core_df.id)]
    
    # concatenate core and branch senses
    extended_df = pd.concat([core_df,branches_df_red])
    
    # refine the provenance type
    # if the last semantic class id is not equal to provenance
    # this row is a child or descendant
    check_membership = lambda row : row.provenance in row.semantic_class_last_id
    extended_df.loc[(~extended_df.apply(check_membership,axis=1)) & (extended_df.provenance_type=='branch'),
                ["provenance_type"]]  = "branch_descendant"
    
    # save information
    extended_df.to_pickle(f'{dp}/senses_{lemma_id}_extended.pickle')
    return extended_df